In [15]:
!python3.11 -m pip install --upgrade pip -q
!pip install mediapy diffusers transformers accelerate -q

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import mediapy as media
import numpy as np
from diffusers import DDIMScheduler, StableDiffusionPipeline
from Stable_Diffusion import DiffusionPipelineUtils, FLIPD
import json

In [2]:
# load model
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model_id = "CompVis/stable-diffusion-v1-4"

pipe = StableDiffusionPipeline.from_pretrained(model_id)
pipe = pipe.to(device)
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
stable_diffusion = DiffusionPipelineUtils(pipe)
flipd_utils = FLIPD(pipeline=stable_diffusion, device=device)

In [ ]:
prompt = "A painting of a cat in the style of Picasso"
height = pipe.unet.config.sample_size * pipe.vae_scale_factor
width =  pipe.unet.config.sample_size * pipe.vae_scale_factor
num_channels_latents = pipe.unet.config.in_channels
num_images_per_prompt = 1
batch_size = 1

flipd_hist = []
for t in [100, 10, 1]: 
    t_0 = torch.tensor([t])
    x = stable_diffusion.prepare_latents(
        t_0, batch_size=batch_size, num_channels_latents=num_channels_latents, 
        latent_height=height, 
        latent_width=width, 
        latent_dtype=torch.float32
    )

    flipd_score = flipd_utils.flipd(
        t_0, x, guidance_scale=7.5, prompt=prompt
    )
    
    print(flipd_score)
    flipd_hist.append(flipd_score)